# DPO Trainer

- model , model_refence 2개의 모델을 통해서 DPO 방식을 통해 학습하는 코드
- SFT, FFT 을 통해서 만들어진 Base model 에 필요 Task에 맞게 강화학습을 진행
  - 예) safety, helpfullness 와 같은 특정 모델에 맞게 학습
- PPO 방식과 다르게 평가 모델을 따로 개발하지 않고, reference 모델을 두고서 **Chosen / Reject** 방식으로 데이터를 구분해가며 학습하는 방식


### PPO 방식의 문제?
- 기존 방식은 대답을 rank로 분류해서 loss 계산해서 학습하거나 사용. DPO 처럼 Chosen, Rejected 의 방식이 아님
- 문제 )
  -  평가모델에 대한 성능 평가는 어떻게 할 것인가?, 평가 모델은 오버피팅 해두는 것이 맞는가?... 
  - 평가 모델에 대한 성능이 좋아야 함. 보통 GPT4 도 많이 사용
  - 평가 모델을 위한 학습 데이터셋 구축이 필요 -> 한국 데이터의 경우 양질의 데이터가 많지 않음
  - 평가모델 학습 방법은??

## Lib

In [11]:
import os
import torch
from tqdm import tqdm 
import random
import gc

from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset, load_dataset
from trl import SFTTrainer, DPOTrainer, DataCollatorForCompletionOnlyLM
from peft import PeftModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model

## BitsAndBytes

In [12]:
output_dir = ""
model_name = ""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_typr="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization=bnb_config, device_map="auto")
model.config.use_cache=False
model = prepare_model_for_kbit_training(model)

model_ref = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## Data

In [ ]:
data_file = {"train":""}
dataset = load_dataset("json", data_files=data_file, split="train")
print(f"Dataset size : {len(dataset)}")

In [ ]:
dataset

In [ ]:
# DATA 검수
random.seed(42)
temp = random.sample(list(dataset), 1000)
# temp[:2]

sampled_dataset = Dataset.from_list(temp)

In [19]:
def return_prompt_response(samples):
    return {
        "prompt" : [
            f"{system}\n### Input: ```{question}```\n### Output: "
            for (system, question) in zip(samples['system'], samples['question'])
        ],
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }
    

In [ ]:
origin_columns = sampled_dataset.origin_columns

sampled_dataset = sampled_dataset.map(
    return_prompt_response,
    batched=True,
    remove_columns=origin_columns
)

# sampled_dataset 확인

## PEFT
- 파라미터의 일부 값만 수정하는 학습 방식.

In [ ]:
def print_trainable_param(model):
    """
    print the number of trainable param in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"trainable params : {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

In [ ]:
peft_config = LoraConfig(
    r = 64,
    lora_alpha=128,
    target_modules=["q_proj","k_proj","v_proj","o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CASUAL_LM"
)

In [ ]:
model = get_peft_model(model, peft_config)
print_trainable_param(model)

## Training

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=3,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    save_steps=50,
    logging_steps=10,
    bf16=True,
    output_dir=output_dir,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    remove_unused_columns=False
)

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=sampled_dataset,
    tokenizer=tokenizer,
    max_prompt_length=2048,
    max_length=4096,
    
)

dpo_trainer.train()
dpo_trainer.save_model(output_dir)

output_dir = os.path.join(output_dir, "final_checkpoint")
dpo_trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

## Cleaning

In [ ]:
del dpo_trainer, model, model_ref
gc.collect()
torch.cuda.empty_cache()